In [ ]:
%matplotlib inline
from core.triangulation_calibration_module import Calibration, Triangulation_Positions, Triangulation_Recordings

In [ ]:
a = Calibration(calibration_directory = 'test_data/220826_Calibration/', 
                project_config_filepath = 'test_data/project_config.yaml',
                recording_config_filepath = 'test_data/recording_config.yaml',
                overwrite = False, load_calibration = True, max_frame_count = 300, use_gpu = False, output_directory = 'test_data/220826_Calibration/')

In [ ]:
a.synchronization_crossvalidation.plot()

In [ ]:
for individual_plot in a.synchronization_individuals:
    individual_plot.plot()
for individual_plot in a.led_detection_individuals:
    individual_plot.plot()

In [ ]:
a.run_calibration(verbose = True)

In [ ]:
b = Triangulation_Recordings(recording_directory = 'test_data/220826_Recording/', 
                             calibration_toml_filepath = 'test_data/220826_Calibration/calibration_220826.toml', 
                             recording_config_filepath = 'test_data/recording_config.yaml', 
                             project_config_filepath= 'test_data/project_config.yaml',
                             output_directory = "test_data/220826_Recording/", overwrite = False, load_calibration=True, use_gpu = False)

In [ ]:
b.run_triangulation()

In [ ]:
b.df

In [ ]:
c = Triangulation_Positions(positions_directory = 'test_data/220826_Calibration/', 
                            calibration_toml_filepath = 'test_data/220826_Calibration/calibration_220826.toml', 
                            recording_config_filepath = 'test_data/recording_config.yaml', 
                            project_config_filepath= 'test_data/project_config.yaml',
                            output_directory = "test_data/220826_Calibration/", overwrite = False, load_calibration=True)

In [ ]:
c.get_marker_predictions()

In [ ]:
c.run_triangulation()

In [ ]:
#ToDo: # distinguish between max_frame_count! (used in intrinsic_calibrations and save_videoto_disk)

## dependencies

In [ ]:
"""
pip install aniposelib
pip install ffmpeg
pip install ffmpeg-python
pip install imageio-ffmpeg
jupyter lab
"""